### Project 02: Create a GUI Notebook Program

enter your name and date here

Project 2 will adapt the procedural code we have been working on in INST326_SimpleGUI_Note_Form_IO.ipynb to create an OOP version of the program using three classes.

    A Notebook or MainWindow class
    A Form or TopWindow class
    A Note class

The MainWindow class is a subclass of Tkinter’s tk.Tk class and thus inherits its attributes and methods, while allowing us to customize the new window objects to our needs. These new window objects will represent “notebooks” or collections of notes, and allow us to work with those notes. (I have named it MainWindow because this class definition could be used to create any kind of main window in Tkinter. We are using it to represent notebooks in this application, but you might use it for other purposes in onther applications.)


The TopWindow class creates a new top window in Tkinter, which is essentially a form for entering the title, text, links, and tags for our note. When we submit the note, this “form” object has a method that creates the note’s metadata and a new Note object. That note object is appended to the list of all notes, which is an attribute of the notebook (MainWindow) class.
The Note class creates note objects that contain the  title, text, links, tags, and metadata for each note.

For Project 02 you will:  

    1. Create one notebook or MainWindow object  
    2. Create several (at least 3) ‘real’ notes for your final submission. By ‘real’ I mean actual notes about python that are useful to you. Print them in the cell at the bottom of the notebook.
    3. Save those notes to a single .txt, .csv, or .json file (your choice of format).  
    4. Retrieve those notes and 
    5. Display representations of them as either labels or buttons in the  main window (remember how we displayed cards in project 01). These representations are not whole notes. Rather, they are object representations of the notes.  
    6. When they are clicked the whole note pops up in a new window - either the form window or a ‘read-only’ version of the form window.



#### Complete your code in the cell below

The code cell below contains the imports you will need; the top level structure for the three classes to get you started; and the execution code at the bottom. 

In [5]:
# imports
import tkinter as tk
from tkinter import ttk
import datetime # one module for working with dates and times

   


# The MainWindow class creates a custom GUI window based on the tkinter window (tk.Tk)
# It has an __init__() method, and three additional methods (new_note(), open_notebook(), and save_notebook())
# These methods correspond to new, open, and save buttons in the window.
# The new_note method calls the NoteForm class to create a new note form top level window.

class MainWindow(tk.Tk):
    def __init__(self):  #initialize the main window
        super().__init__() # initialize it as a tkinter window
        
        self.geometry("600x400") # set the default window size
        self.title('Notebook') # set the default window title
        self.notebook = [] # initialize an attribute named 'notebook' as an empty list
        self.current_note = None

        create_button = tk.Button(self, text='Create New Note', command=self.new_note)
        create_button.grid(row=0, column=0, padx=10, pady=10)

        save_button = tk.Button(self, text='Save File', command=self.save_notebook)
        save_button.grid(row=0, column=1, padx=10, pady=10)

        quit_button = tk.Button(self, text='Quit Application', command=self.destroy)
        quit_button.grid(row=0, column=2, padx=10, pady=10)

        title_label = tk.Label(self, text='Notes')
        title_label.grid(row=1, column=0, padx=2, pady=2)

        self.read_notebook_file()
        self.refresh_notes_list()

        self.mainloop()   

    def read_notebook_file(self):
        file = open('INST326_NOTEBOOK.txt', 'r')
        notes = file.readlines()
        for note in notes:
            fields = note.split('\t')
            if len(fields) < 4:
                continue
            new_note_dict = {'title': fields[0], 'text': fields[1], 'links': fields[2], 'tags': fields[3]}
            new_note = Note(new_note_dict)
            self.notebook.append(new_note)

    def refresh_notes_list(self):
        i = 2
        for note in self.notebook:
            self.current_note = note
            note_button = tk.Button(self, text=note.title, command=self.open_notebook)
            note_button.grid(row=i, column=0, padx=2, pady=2, sticky='E')
            i += 1

    def new_note(self):
        note_window = NoteForm(self, self.notebook, None)
        return None

    def open_notebook(self):
        note_window = NoteForm(self, self.notebook, self.current_note)
        return None

    def save_notebook(self):
        with open("INST326_NOTEBOOK.txt", "w") as f:
            for notes in self.notebook:
                f.write(f"\n{notes.title}\t{notes.text}\t{notes.links}\t{notes.tags}")
        return None

# the NoteForm() class creates a Toplevel window that is a note form containing fields for
# data entry for title, text, link, and tags. It also calculates a meta field with date, time, and timezone
# the Noteform class has an __init__() method, and a submit() method that is called by a submit button
# the class may contain additional methods to perform tasks like calculating the metadata, for example
# the submit method calls the MakeNote class that transforms the the entered data into a new note object.

class NoteForm(tk.Toplevel):
    def __init__(self, master, notebook, note): # initialize the new object
        super().__init__(master) # initialize it as a toplevel window
        self.notebook = notebook 

        # title
        title_label = tk.Label(self, bg='light gray', text='Note Title:')
        self.note_title = tk.Entry(self, width=80)

        title_label.grid(row=0, column=0, padx=2, pady=2, sticky='E')
        self.note_title.grid(row=0, column=1, padx=10, pady=2, sticky='W')

        # text
        text_label = tk.Label(self, bg='light gray', text='Note Text:')
        self.note_text = tk.Text(self, height=5, width=60)

        text_label.grid(row=1, column=0, padx=2, pady=2, sticky='E')
        self.note_text.grid(row=1, column=1, padx=10, pady=2, sticky='W')

        # links
        links_label = tk.Label(self, bg='light gray', text='Note Links:')
        self.note_links = tk.Entry(self, width=80)

        links_label.grid(row=2, column=0, padx=2, pady=2, sticky='E')
        self.note_links.grid(row=2, column=1, padx=10, pady=10, sticky='W')

        # tags
        tags_label = tk.Label(self, bg='light gray', text='Note Tags:')
        self.note_tags = tk.Entry(self, width=80)

        tags_label.grid(row=3, column=0, padx=2, pady=2, sticky='E')
        self.note_tags.grid(row=3, column=1, padx=10, pady=2, sticky='W')

        # submit button
        submit_button = tk.Button(self, text='Submit', command=self.submit)

        # if viewing an existing note, remove submit button and disable inputs
        if note is None:
            submit_button.grid(row=4, column=1, padx=10, pady=10, sticky='W')
        else:
            self.note_title.insert(0, note.title)
            self.note_title.config(state=tk.DISABLED) 

            self.note_text.insert("1.0", note.text)
            self.note_text.config(state=tk.DISABLED) 

            self.note_links.insert(0, note.links) 
            self.note_links.config(state=tk.DISABLED) 

            self.note_tags.insert(0, note.tags) 
            self.note_tags.config(state=tk.DISABLED) 
  
    def submit(self):
        new_note_dict = {'title': self.note_title.get(), 'text': self.note_text.get("1.0", "end-1c"), 'links': self.note_links.get(), 'tags': self.note_tags.get()}
        new_note = Note(new_note_dict)
        self.notebook.append(new_note)
        self.destroy()
        self.master.refresh_notes_list()
        return None

    
# The Note class takes a dictionary containing the data entered into the form window,
# and transforms it into a new note object.
# At present the note objects have attributes but no methods.
class Note():
    def __init__(self):
        self.title = ""
        self.text = ""
        self.links = ""
        self.tags = ""

    def __init__(self, note_dict):
        self.title = note_dict["title"] 
        self.text = note_dict["text"] 
        self.links = note_dict["links"] 
        self.tags = note_dict["tags"] 




    
# main execution
if __name__ == '__main__':
    main_window = MainWindow() # this creates a notebook / main window called main_window. You may change the name if you want
    main_window.mainloop()

#### Print your three notes below

In [6]:
# print your notes here

file = open('INST326_NOTEBOOK.txt', 'r')
notes = file.readlines()
for note in notes:
    fields = note.split('\t')
    if len(fields) < 4:
        continue
    new_note_dict = {'title': fields[0], 'text': fields[1], 'links': fields[2], 'tags': fields[3]}
    print(new_note_dict)


{'title': 'GUI_and_Tkinter', 'text': 'Remember to import Tkinter as TK, then create a main window and create buttons and labels! Using function Tk()is useful for buttons, labels, interface and more user interaction boxes!', 'links': 'https://realpython.com/python-gui-tkinter/', 'tags': '#userinteraction #GUI #Tkinter #widgets\n'}
{'title': 'Read_files', 'text': "Going back to the basics of INST126, we can open, read and print a files text. To do this, we start with #with open to open the txt. #with open('something.txt', 'r') as file (#r is used to read.) then we can use a for loop to read and print the text, like this #for line in file print(this)", 'links': 'https://www.freecodecamp.org/news/python-open-file-how-to-read-a-text-file-line-by-line/', 'tags': '#forloops #r #withopen #fileIO\n'}
{'title': 'Networked_programs', 'text': "will need TCP port, socket and HTTP. Add a socket to establish a network connection. #import socket and create the object #ex_socket = socket.socket(socket.